# Artificial Neural Network

### Importing the libraries

In [4]:
# Import necessary libraries
!pip install scikit-learn
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
print(tf.__version__)

2.16.2


## Part 1 - Data Preprocessing

Import dataset and assign to variables.

In [6]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [7]:
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


Encoding 'Gender'

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [9]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


Encoding 'Geography' (one hot encoding)
France -> 1 0 0
Spain -> 0 1 0
Germany -> 0 0 1

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [11]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Splitting the dataset into the training and test dataset

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [13]:
X_train.shape

(8000, 12)

In [14]:
X_train.shape
X_test.shape
y_train.shape
y_test.shape

(2000,)

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

In [59]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=128, activation='relu'))
ann.add(tf.keras.layers.Dense(units=128, activation='relu'))
ann.add(tf.keras.layers.Dense(units=64, activation='relu'))
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

In [60]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [78]:
# Train the ANN
history = ann.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

# Evaluate the model on train and test sets
train_loss, train_accuracy = ann.evaluate(X_train, y_train)
test_loss, test_accuracy = ann.evaluate(X_test[:2000], y_test[:2000])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 856us/step - accuracy: 0.7072 - loss: 0.6068 - val_accuracy: 0.7975 - val_loss: 0.4734
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.7959 - loss: 0.4685 - val_accuracy: 0.7975 - val_loss: 0.4416
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - accuracy: 0.7979 - loss: 0.4431 - val_accuracy: 0.8065 - val_loss: 0.4278
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - accuracy: 0.8059 - loss: 0.4299 - val_accuracy: 0.8105 - val_loss: 0.4193
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.8171 - loss: 0.4147 - val_accuracy: 0.8145 - val_loss: 0.4098
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - accuracy: 0.8173 - loss: 0.4155 - val_accuracy: 0.8245 - val_loss: 0.3981
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - accuracy: 0.8301 - loss: 0.3946 - val_accuracy: 0.8425 - val_loss: 0.3821
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.8432 - loss: 0

In [79]:
print(f"Training accuracy: {train_accuracy}")
print(f"Test accuracy: {test_accuracy}")
print(f"=========================")
print(f"Training loss: {train_loss}")
print(f"Test loss: {test_loss}")

Training accuracy: 0.8632500171661377
Test accuracy: 0.8610000014305115
Training loss: 0.3335702121257782
Test loss: 0.33949699997901917


In [80]:
# Evaluate the model on the test set
test_loss, test_accuracy = ann.evaluate(X_test, y_test)
train_loss, test_accuracy = ann.evaluate(X_train, y_train)

# Print the accuracy
print(f"Accuracy on test set: {test_accuracy:.2f}")
print(f"Loss : {train_loss:.2f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step - accuracy: 0.8649 - loss: 0.3306
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 457us/step - accuracy: 0.8655 - loss: 0.3278
Accuracy on test set: 0.86
Loss : 0.33


In [81]:
# Predict on test data
y_pred = (ann.predict(X_test)> 0.5).astype("int32")

classification_rep = classification_report(y_test[:2000], y_pred, output_dict=True)

accuracy = classification_rep['accuracy']
loss = train_loss # Assuming you want recall for class '0'

print(f"Accuracy: {accuracy:.2f}")
#print(f"Loss : {loss:.2f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step
Accuracy: 0.86


In [82]:
# Predict on test data
y_pred = (ann.predict(X_test[:2000]) > 0.5).astype("int32")

# Print classification report
print("Classification Report:")
print(classification_report(y_test[:2000], y_pred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.94      0.92      1595
           1       0.70      0.55      0.62       405

    accuracy                           0.86      2000
   macro avg       0.80      0.75      0.77      2000
weighted avg       0.85      0.86      0.85      2000



In [83]:
# Show predictions and actual values
predictions = (ann.predict(X_test) > 0.5).astype(int)
print("Predictions on the test set:")
print(predictions[:10].flatten())

print("Actual labels on the test set:")
print(y_test[:10])

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step
Predictions on the test set:
[0 0 0 0 0 1 0 0 0 1]
Actual labels on the test set:
[0 1 0 0 0 1 0 0 1 1]


In [84]:
ann.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 6)              │            78 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 383 (1.50 KB)

 Trainable params: 127 (508.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 256 (1.00 KB)

In [85]:
# Save the entire model to a HDF5 file
ann.save('churn_model.h5')
print("Model saved successfully.")

Model saved successfully.


In [69]:
import tensorflow as tf
import os

# Install necessary libraries
!pip install pyyaml h5py

# Assume X_train, y_train, X_test, y_test are defined and ann is the trained model

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path + '.weights.h5',
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
ann.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[cp_callback])

# List checkpoint files
print(os.listdir(checkpoint_dir))


Defaulting to user installation because normal site-packages is not writeable
Epoch 1/10
234/250 ━━━━━━━━━━━━━━━━━━━━ 0s 655us/step - accuracy: 0.9960 - loss: 0.0170


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



Epoch 1: saving model to training_1/cp.ckpt.weights.h5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9958 - loss: 0.0176 - val_accuracy: 0.8090 - val_loss: 1.7567
Epoch 2/10
222/250 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - accuracy: 0.9765 - loss: 0.0724
Epoch 2: saving model to training_1/cp.ckpt.weights.h5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - accuracy: 0.9750 - loss: 0.0770 - val_accuracy: 0.8070 - val_loss: 1.4703
Epoch 3/10
223/250 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - accuracy: 0.9784 - loss: 0.0617
Epoch 3: saving model to training_1/cp.ckpt.weights.h5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9788 - loss: 0.0607 - val_accuracy: 0.8160 - val_loss: 1.5072
Epoch 4/10
212/250 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - accuracy: 0.9908 - loss: 0.0307
Epoch 4: saving model to training_1/cp.ckpt.weights.h5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9907 - loss: 0.0305 - val_accuracy: 0.8090 - val_loss: 1.5117
Epoch 5/10
211/250 ━━━━━━━━━━━━━━━━━━━━ 0s 72

In [86]:
import os

# Assuming checkpoint_dir is defined after training
print("Checkpoint files:")
print(os.listdir(checkpoint_dir))

Checkpoint files:
['cp.ckpt.weights.h5']


In [88]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Load dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

# Encode categorical data
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Load the saved model
loaded_model = tf.keras.models.load_model('churn_model.h5')

# Evaluate the loaded model
train_loss, train_accuracy = loaded_model.evaluate(X_train, y_train, verbose=0)
test_loss, test_accuracy = loaded_model.evaluate(X_test, y_test, verbose=0)

print(f"Training accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"=========================")
print(f"Training loss: {train_loss:.4f}")
print(f"Test loss: {test_loss:.4f}")

# Evaluate the model on the test set
test_loss, test_accuracy = ann.evaluate(X_test, y_test)
train_loss, test_accuracy = ann.evaluate(X_train, y_train)

# Print the accuracy
print(f"Accuracy on test set: {test_accuracy:.2f}")
print(f"Loss : {train_loss:.2f}")


Training accuracy: 0.8633
Test accuracy: 0.8610
Training loss: 0.3336
Test loss: 0.3395
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 529us/step - accuracy: 0.8649 - loss: 0.3306
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - accuracy: 0.8655 - loss: 0.3278
Accuracy on test set: 0.86
Loss : 0.33


#งานที่ 1: ให้สร้างข้อมูลทดสอบ 5 ชุดที่แตกต่างกันเพื่อทำ Prediction จากโมเดลของตัวเอง

In [91]:
t(sc.transform([[3, 1, 1, 650, 0, 35, 5, 75000, 2, 1, 1, 80000]])) > 0.5)
print(ann.predict(sc.transform([[4, 0, 0, 720, 1, 55, 10, 150000, 1, 1, 0, 120000]])) > 0.5)
print(ann.predict(sc.transform([[5, 1, 0, 580, 1, 28, 1, 5000, 3, 0, 1, 45000]])) > 0.5)
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
print(ann.predict(sc.transform([[22, 0, 0, 497, 1, 44, 1, 125510, 2, 0, 1, 190857]])) > 0.5)
print(ann.predic

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[False]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[ True]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[[False]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[ True]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[False]]


#งานที่ 2: Predicting the Test set results ด้วยกลุ่มคำสั่ง

In [92]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


งานที่ 3: สร้าง Confusion Matrix ด้วยกลุ่มคำสั่ง

In [93]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1498   97]
 [ 181  224]]


0.861